This notebook uses lightGBM to make predictions.

We use the following features
* playerId
* position
* teamId(rosters)
* status(rosters)
* playerBoxScores

and the date 20200401~20200431 as the validation data.

But I think there is room for improvement.  
If you have better ways, I would appreciate it if you could comment on it.

このnotebookではlightGBMを使って予測します。

特徴量は以下のものを使用しています。
* playerId
* position
* teamId(rosters)
* status(rosters)
* playerBoxScores

20200401~20200431を日時をvalidation dataとしていますが、一考の余地がありそうです。  
もし良さそうな方法があればコメントしていただけると幸いです。

https://www.kaggle.com/columbia2131/mlb-lightgbm-starter-dataset-code-en-ja

## About Dataset

Train.csv is stored as a csv file with each column as follows.  

train.csvを以下のようにして各カラムをcsvファイルとして保管しています。

In [ ]:
"""
%capture

!pip install pandarallel 

import gc

import numpy as np
import pandas as pd
from pathlib import Path

from pandarallel import pandarallel
pandarallel.initialize()

BASE_DIR = Path('../input/mlb-player-digital-engagement-forecasting')
train = pd.read_csv(BASE_DIR / 'train.csv')

null = np.nan
true = True
false = False

for col in train.columns:

    if col == 'date': continue

    _index = train[col].notnull()
    train.loc[_index, col] = train.loc[_index, col].parallel_apply(lambda x: eval(x))

    outputs = []
    for index, date, record in train.loc[_index, ['date', col]].itertuples():
        _df = pd.DataFrame(record)
        _df['index'] = index
        _df['date'] = date
        outputs.append(_df)

    outputs = pd.concat(outputs).reset_index(drop=True)

    outputs.to_csv(f'{col}_train.csv', index=False)
    outputs.to_pickle(f'{col}_train.pkl')

    del outputs
    del train[col]
    gc.collect()
"""

## Training

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error
from datetime import timedelta
from functools import reduce
from tqdm import tqdm
import lightgbm as lgbm
import mlb
import pickle
from datetime import datetime as dt
import copy
from sklearn.decomposition import PCA
import math
import gc

In [ ]:
BASE_DIR = Path('../input/c/mlb-player-digital-engagement-forecasting')
TRAIN_DIR = Path('../input/mlb-pdef-train-dataset')

In [ ]:
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

In [ ]:
players = pd.read_csv(BASE_DIR / 'players.csv')

rosters = pd.read_pickle(TRAIN_DIR / 'rosters_train.pkl')
targets = pd.read_pickle(TRAIN_DIR / 'nextDayPlayerEngagement_train.pkl')
scores1 = pd.read_pickle(TRAIN_DIR / 'playerBoxScores_train.pkl')
scores = scores1.groupby(['playerId','date']).sum().reset_index()
twitter = pd.read_pickle(TRAIN_DIR / "playerTwitterFollowers_train.pkl")
games = pd.read_pickle(TRAIN_DIR / 'games_train.pkl')
events = pd.read_pickle(TRAIN_DIR / 'events_train.pkl')
standings = pd.read_pickle(TRAIN_DIR / 'standings_train.pkl')
teamtwitter = pd.read_pickle(TRAIN_DIR / 'teamTwitterFollowers_train.pkl')
transaction = pd.read_pickle(TRAIN_DIR / 'teamTwitterFollowers_train.pkl')
awards = pd.read_csv(BASE_DIR / 'awards.csv')
seasons = pd.read_csv(BASE_DIR / 'seasons.csv')
teams = pd.read_csv(BASE_DIR / 'teams.csv')
player_target_stats = pd.read_csv("../input/old-pts/player_target_stats.csv")
gc.collect()

In [ ]:
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

In [ ]:
games = pd.read_pickle(TRAIN_DIR / 'games_train.pkl')
games[games.duplicated(subset=["gamePk"], keep=False)]
#games.index[games["detailedGameState"] == "Postponed"].shape
#games.drop(games.loc[games['detailedGameState']=='Postponed'].index, inplace=True)
#games["detailedGameState"].unique()
#games[games.duplicated(subset=["gamePk"], keep=False)]
#games['detailedGameState']=='Postponed']だとスコアはキロクされていない
games

In [ ]:
awards2 = awards.groupby("playerId").count()
awards2 = awards2.reset_index()

In [ ]:
teamtwitter["teamnumberOfFollowers"] = teamtwitter["numberOfFollowers"]
teamtwi = teamtwitter.groupby("teamId").mean()["teamnumberOfFollowers"].reset_index()
teamtwi

In [ ]:
targets_cols = ['playerId', 'target1', 'target2', 'target3', 'target4', 'date']
players_cols = ['playerId', 'primaryPositionName']
rosters_cols = ['playerId', 'teamId', 'status', 'date']
yesterday_targets_cols = ["date","playerId","yest_target1","yest_target2","yest_target3","yest_target4"]
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances', 'date',"gamePk"]
games_cols = ["gamePk","homeId","dayNight","seriesDescription","gamesInSeries","homeWinner","awayWinner","homeScore","awayScore","gameType",
              "gameDate"]
playertwitter_cols = ["playerId","numberOfFollowers"]
awards_cols = ["playerId","awardName"]
standings_cols = ["date","teamId","divisionRank","divisionLeader","wildCardLeader","leagueRank","divisionId","gameDate"]
teamtwitter_cols = ["teamId","teamnumberOfFollowers"]
targets["engagementMetricsDate"] = targets["engagementMetricsDate"].str.replace('-', '')
yesterday_targets = targets.drop('date', axis=1)
yesterday_targets = yesterday_targets.rename(columns={'engagementMetricsDate':'date', 'target1': 'yest_target1','target2': 'yest_target2','target3': 'yest_target3','target4': 'yest_target4'})
yesterday_targets["date"] = yesterday_targets["date"].astype(int)
yesterday_targets["date"]

In [ ]:
feature_cols = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored','homeRuns',
       'strikeOuts', 'baseOnBalls',  'hits', 'hitByPitch',
       'atBats', 'stolenBases', 
       'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacFlies',
       'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching','winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching',  'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored',
       'sacFliesPitching', 'saves', 'holds',
       'assists', 'putOuts', 'errors', 'chances','target1_mean',
 'target1_median',
 'target1_std',
 'target1_min',
 'target1_max',
 'target1_prob',
 'target2_mean',
 'target2_median',
 'target2_std',
 'target2_min',
 'target2_max',
 'target2_prob',
 'target3_mean',
 'target3_median',
 'target3_std',
 'target3_min',
 'target3_max',
 'target3_prob',
 'target4_mean',
 'target4_median',
 'target4_std',
 'target4_min',
 'target4_max',
 'target4_prob',"divisionId","teamnumberOfFollowers","preseasonhits","preseasonhomerun"]
feature_cols2 = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored',# 'doubles', 'triples', 'homeRuns',
       'baseOnBalls', 'hits',
      # 'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'plateAppearances', 'totalBases', 'rbi',
      # 'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'winsPitching',
       'lossesPitching',# 'flyOutsPitching', 'airOutsPitching',
       'runsPitching', 
       'strikeOutsPitching',
       #'hitsPitching',
       'hitByPitchPitching',  'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched',
        'battersFaced', 
        'balks', 'pickoffsPitching',
        'inheritedRunners',
        
       #'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'putOuts','chances','target1_mean',
 'target1_median',
 'target1_std',
 'target1_min',
 'target1_max',
 'target1_prob',
 'target2_mean',
 'target2_median',
 'target2_std',
 'target2_min',
 'target2_max',
 'target2_prob',
 'target3_mean',
 'target3_median',
 'target3_std',
 'target3_min',
 'target3_max',
 'target3_prob',
 'target4_mean',
 'target4_median',
 'target4_std',
 'target4_min',
 'target4_max',
 'target4_prob',
    'target1',"divisionId","teamnumberOfFollowers","preseasonhits","preseasonhomerun"]
feature_cols3 = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', #'gamesPlayedBatting', 'flyOuts',
        'homeRuns',
      # 'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
     # 'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
        'totalBases', 'rbi',
      # 'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
        'gamesStartedPitching',
      # 'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 
      # 'groundOutsPitching', 'runsPitching', 'doublesPitching',
      # 'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       #'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       #'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'inningsPitched', 
        'battersFaced', 'pitchesThrown', 
      # 'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
      # 'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       #'inheritedRunnersScored', 'catchersInterferencePitching',
       #'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
      # 'assists', 'putOuts', 'errors', 'chances',
      'target1_mean',
 'target1_median',
 'target1_std',
 'target1_min',
 'target1_max',
 'target1_prob',
 'target2_mean',
 'target2_median',
 'target2_std',
 'target2_min',
 'target2_max',
 'target2_prob',
 'target3_mean',
 'target3_median',
 'target3_std',
 'target3_min',
 'target3_max',
 'target3_prob',
 'target4_mean',
 'target4_median',
 'target4_std',
 'target4_min',
 'target4_max',
 'target4_prob',
    "divisionId","teamnumberOfFollowers","target2","preseasonhits","preseasonhomerun"]
feature_cols4 = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
       'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
        'runsScored','triples','groundIntoDoublePlay',
       'strikeOuts', 'baseOnBalls','hits', 
       'atBats', 'caughtStealing',
       'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies',
        'gamesPlayedPitching',
        'winsPitching',
       'airOutsPitching',
       'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
        'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 
        'inningsPitched', 'saveOpportunities',
       'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes','wildPitches', 
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 
       'sacBuntsPitching', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances','target1_mean',
 'target1_median',
 'target1_std',
 'target1_min',
 'target1_max',
 'target1_prob',
 'target2_mean',
 'target2_median',
 'target2_std',
 'target2_min',
 'target2_max',
 'target2_prob',
 'target3_mean',
 'target3_median',
 'target3_std',
 'target3_min',
 'target3_max',
 'target3_prob',
 'target4_mean',
 'target4_median',
 'target4_std',
 'target4_min',
 'target4_max',
 'target4_prob',
 'target1', "divisionId","target2","target3","teamnumberOfFollowers","preseasonhits","preseasonhomerun"]

In [ ]:
train = targets[targets_cols].merge(players[players_cols], on=['playerId'], how='left')
train = train.merge(rosters[rosters_cols], on=['playerId', 'date'], how='left')
train = train.merge(scores[scores_cols], on=['playerId', 'date'], how='left')
train = train.merge(games[games_cols], on=["gamePk"], how="left")
train = train.merge(standings[standings_cols], on=['date',"teamId"], how='left')
train = train.merge(teamtwi[teamtwitter_cols], on=['teamId'], how='left')
train = train.merge(player_target_stats, how='inner', left_on=["playerId"],right_on=["playerId"])

In [ ]:
# label encoding
player2num = {c: i for i, c in enumerate(train['playerId'].unique())}
position2num = {c: i for i, c in enumerate(train['primaryPositionName'].unique())}
teamid2num = {c: i for i, c in enumerate(train['teamId'].unique())}
status2num = {c: i for i, c in enumerate(train['status'].unique())}
daynight2num = {c: i for i, c in enumerate(train['dayNight'].unique())}
seriesDescription2num = {c: i for i, c in enumerate(train['seriesDescription'].unique())}
gameType2num = {c: i for i, c in enumerate(train['gameType'].unique())}
train['label_playerId'] = train['playerId'].map(player2num)
train['label_primaryPositionName'] = train['primaryPositionName'].map(position2num)
train['label_teamId'] = train['teamId'].map(teamid2num)
train['label_status'] = train['status'].map(status2num)
train["label_daynight"] = train['dayNight'].map(daynight2num)
train["label_seriesDescription"] = train["seriesDescription"].map(seriesDescription2num)

In [ ]:
twitter = twitter.groupby("playerId").mean().reset_index()
twitter


In [ ]:
train_2017post = train[train["date"]<=20180328]
train_2018reg = train[(train["date"]>=20180329)&(train["date"]<=20181001)]
train_2018post = train[(train["date"]>=20181002)&(train["date"]<=20190319)]
train_2019reg = train[(train["date"]>=20190320)&(train["date"]<=20190929)]
train_2019post = train[(train["date"]>=20190930)&(train["date"]<=20200722)]
train_2020reg = train[(train["date"]>=20200723)&(train["date"]<=20200927)]
train_2020post = train[(train["date"]>=20200928)&(train["date"]<=20210331)]
train_2021reg = train[(train["date"]>=20210401)&(train["date"]<=20211003)]


In [ ]:
#make feature
hitscol = ["playerId","preseasonhits"]
hits_2018 = train_2018reg.groupby("playerId").sum()["hits"].reset_index()
hits_2018["preseasonhits"] = hits_2018["hits"]
train_2018post = train_2018post.merge(hits_2018[hitscol], on=['playerId'], how='left')
train_2019reg = train_2019reg.merge(hits_2018[hitscol], on=['playerId'], how='left')

hits2019col = ["playerId","hits2019"]
hits_2019 = train_2019reg.groupby("playerId").sum()["hits"].reset_index()
hits_2019["preseasonhits"] = hits_2019["hits"]
train_2019post = train_2019post.merge(hits_2019[hitscol], on=['playerId'], how='left')
train_2020reg = train_2020reg.merge(hits_2019[hitscol], on=['playerId'], how='left')

hits2020col = ["playerId","hits2020"]
hits_2020 = train_2020reg.groupby("playerId").sum()["hits"].reset_index()
hits_2020["preseasonhits"] = hits_2020["hits"]
train_2020post = train_2020post.merge(hits_2020[hitscol], on=['playerId'], how='left')
train_2021reg = train_2021reg.merge(hits_2020[hitscol], on=['playerId'], how='left')
del hits_2018,hits_2019
gc.collect()


gamescol = ["playerId","preseasonhomerun"]
game2018 = train_2018reg.groupby("playerId").sum()["homeRuns"].reset_index()
game2018["preseasonhomerun"] = game2018["homeRuns"]
train_2018post = train_2018post.merge(game2018[gamescol], on=['playerId'], how='left')
train_2019reg = train_2019reg.merge(game2018[gamescol], on=['playerId'], how='left')

game2019 = train_2019reg.groupby("playerId").sum()["homeRuns"].reset_index()
game2019["preseasonhomerun"] = game2019["homeRuns"]
train_2019post = train_2019post.merge(game2019[gamescol], on=['playerId'], how='left')
train_2020reg = train_2020reg.merge(game2019[gamescol], on=['playerId'], how='left')

game2020 = train_2020reg.groupby("playerId").sum()["homeRuns"].reset_index()
game2020["preseasonhomerun"] = game2020["homeRuns"]
train_2020post = train_2020post.merge(game2020[gamescol], on=['playerId'], how='left')
train_2021reg = train_2021reg.merge(game2020[gamescol], on=['playerId'], how='left')

del game2018,game2019
gc.collect()

In [ ]:
train = pd.concat([train_2017post,train_2018reg,train_2018post,train_2019reg,train_2019post,train_2020reg,train_2020post,train_2021reg])
train["preseasonhits"]
del train_2017post,train_2018post,train_2019post, train_2020post, train_2018reg, train_2019reg, train_2020reg
gc.collect

In [ ]:

train_y = train[['target1', 'target2', 'target3', 'target4']]


_index = (train['date'] < 20210701)
gc.collect()


x_train = train.loc[_index].reset_index(drop=True)
y_train = train_y.loc[_index].reset_index(drop=True)
x_valid = train.loc[~_index].reset_index(drop=True)
y_valid = train_y.loc[~_index].reset_index(drop=True)



gc.collect()

In [ ]:
"""
def fit_lgbm(x_train, y_train, x_valid, y_valid, params: dict=None, verbose=100):
    oof_pred = np.zeros(len(y_valid), dtype=np.float32)
    #model = lgbm.LGBMRegressor(**params)
    best_params, tuning_history = {}, []
    
    lgb_train = lgbm.Dataset(x_train, y_train)
    lgb_eval = lgbm.Dataset(x_valid, y_valid, reference=lgb_train)
    model = lgbm.train(params, 
        lgb_train, valid_sets=lgb_eval,  
        early_stopping_rounds=100,
        num_boost_round=10000,
        verbose_eval=100,
        #verbose=verbose,               
        )
    oof_pred = model.predict(x_valid)
    score = mean_absolute_error(oof_pred, y_valid)
    print('mae:', score)
    return oof_pred, model, score



import pickle



params1 = {'objective':'mae',
           'reg_alpha': 0.14947461820098767, 
           'reg_lambda': 0.10185644384043743, 
           'n_estimators': 3633,
           'learning_rate': 0.08046301304430488,
           'num_leaves': 674, 
           'feature_fraction': 0.9101240539122566,
           'bagging_fraction': 0.9884451442950513,
           'bagging_freq': 8,
           #'random_state': 1234,
           'min_child_samples': 51,
           }

params2 = {
 'objective':'mae',
 'reg_alpha': 0.1,
 'reg_lambda': 0.1, 
 'n_estimators': 5000,
 'learning_rate': 0.1,
 'random_state': 42,
 "num_leaves": 22,
 
}

params4 = {'objective':'mae',
           'reg_alpha': 0.016468100279441976,
           'reg_lambda': 0.09128335764019105, 
           'n_estimators': 9868, 
           'learning_rate': 0.10528150510326864,
           'num_leaves': 157,
           'feature_fraction': 0.5419185713426886,
           'bagging_fraction': 0.2637405128936662, 
           'bagging_freq': 19, 
           #'random_state': 1234,
           'min_child_samples': 71,
           }


params3 = {
 'objective':'mae',
 'reg_alpha': 0.1,
 'reg_lambda': 0.1, 
 'n_estimators': 10000,
 'learning_rate': 0.1,
 'random_state': 42,
 "num_leaves": 100,
 
}

oof1, model1, score1 = fit_lgbm(
    x_train[feature_cols], y_train['target1'],
    x_valid[feature_cols], y_valid['target1'],
    params1
)

file = '/content/drive/MyDrive/mlb/script/output/model1.pkl'
pickle.dump(model1, open(file, 'wb'))

oof2, model2, score2 = fit_lgbm(
    x_train[feature_cols2], y_train['target2'],
    x_valid[feature_cols2], y_valid['target2'],
    params2
)

file = '/content/drive/MyDrive/mlb/script/output/model2.pkl'
pickle.dump(model2, open(file, 'wb'))

oof3, model3, score3 = fit_lgbm(
    x_train[feature_cols3], y_train['target3'],
    x_valid[feature_cols3], y_valid['target3'],
    params3
)

file = '/content/drive/MyDrive/mlb/script/output/model3.pkl'
pickle.dump(model3, open(file, 'wb'))


oof4, model4, score4 = fit_lgbm(
    x_train[feature_cols4], y_train['target4'],
    x_valid[feature_cols4], y_valid['target4'],
    params4
)


file = '/content/drive/MyDrive/mlb/script/output/model4.pkl'
pickle.dump(model4, open(file, 'wb'))
score = (score1+score2+score3+score4) / 4
print(f'score: {score}')
"""

In [ ]:
model1 = pickle.load(open('../input/mymodels/model1.pkl', 'rb'))
model2 = pickle.load(open('../input/mymodels/model2.pkl', 'rb'))
model3 = pickle.load(open('../input/mymodels/model3.pkl', 'rb'))
model4 = pickle.load(open('../input/mymodels/model4.pkl', 'rb'))

In [ ]:
players_cols = ['playerId', 'primaryPositionName']
rosters_cols = ['playerId', 'teamId', 'status']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances']
standings_cols = ["teamId","divisionRank","divisionLeader","wildCardLeader","leagueRank","divisionId","gameDate"]
yesterday_targets_cols = ["playerId","yest_target1","yest_target2","yest_target3","yest_target4"]
null = np.nan
true = True
false = False
hitscol = ["playerId","preseasonhits"]


## Inference

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
from tqdm import tqdm
import gc
from functools import reduce
from sklearn.model_selection import StratifiedKFold
def make_df(df, col, bool_in=False):
    tp = df.loc[ ~df[col].isnull() ,[col]].copy()
    df.drop(col, axis=1, inplace=True)
    
    tp[col] = tp[col].str.replace("null",'""')
    if bool_in:
        tp[col] = tp[col].str.replace("false",'"False"')
        tp[col] = tp[col].str.replace("true",'"True"')
    tp[col] = tp[col].apply(lambda x: eval(x) )
    a = tp[col].sum()
    gc.collect()
    return pd.DataFrame(a)
#===============
ROOT_DIR = "../input/c/mlb-player-digital-engagement-forecasting"
#UTILITY FUNCTIONS
#=======================#
def flatten(df, col):
    du = (df.pivot(index="playerId", columns="EvalDate", 
               values=col).add_prefix(f"{col}_").
      rename_axis(None, axis=1).reset_index())
    return du
#============================#
def reducer(left, right):
    return left.merge(right, on="playerId")
#========================
TGTCOLS = ["target1","target2","target3","target4"]
def train_lag(df, lag=1):
    dp = df[["playerId","EvalDate"]+TGTCOLS].copy()
    dp["EvalDate"]  =dp["EvalDate"] + timedelta(days=lag) 
    df = df.merge(dp, on=["playerId", "EvalDate"], suffixes=["",f"_{lag}"], how="left")
    return df
#=================================
def test_lag(sub):
    sub["playerId"] = sub["date_playerId"].apply(lambda s: int(  s.split("_")[1]  ) )
    assert sub.date.nunique() == 1
    dte = sub["date"].unique()[0]
    
    eval_dt = pd.to_datetime(dte, format="%Y%m%d")
    dtes = [eval_dt + timedelta(days=-k) for k in LAGS]
    mp_dtes = {eval_dt + timedelta(days=-k):k for k in LAGS}
    
    sl = LAST.loc[LAST.EvalDate.between(dtes[-1], dtes[0]), ["EvalDate","playerId"]+TGTCOLS].copy()
    sl["EvalDate"] = sl["EvalDate"].map(mp_dtes)
    du = [flatten(sl, col) for col in TGTCOLS]
    du = reduce(reducer, du)
    return du, eval_dt
    

 
 

tr = pd.read_csv(f"{ROOT_DIR}/train.csv")
tr = pd.read_csv("../input/mlb-data/target.csv")
print(tr.shape)
gc.collect()

tr["EvalDate"] = pd.to_datetime(tr["EvalDate"])
tr["EvalDate"] = tr["EvalDate"] + timedelta(days=-1)
tr["EvalYear"] = tr["EvalDate"].dt.year
MED_DF = tr.groupby(["playerId","EvalYear"])[TGTCOLS].median().reset_index()
MEDCOLS = ["tgt1_med","tgt2_med", "tgt3_med", "tgt4_med"]
MED_DF.columns = ["playerId","EvalYear"] + MEDCOLS
MED_DF.head()
MAX_LAG = 20
LAGS = list(range(1, MAX_LAG + 1))
FECOLS = [f"{col}_{lag}" for lag in reversed(LAGS) for col in TGTCOLS]


for lag in tqdm(LAGS):
    tr = train_lag(tr, lag=lag)
    gc.collect()
#===========
tr = tr.sort_values(by=["playerId", "EvalDate"])
print(tr.shape)
tr = tr.dropna()
print(tr.shape)
tr = tr.merge(MED_DF, on=["playerId","EvalYear"])
gc.collect()

X = tr[FECOLS+MEDCOLS].values
y = tr[TGTCOLS].values
cl = tr["playerId"].values

NFOLDS = 10
skf = StratifiedKFold(n_splits=NFOLDS)
folds = skf.split(X, cl)
folds = list(folds)
X.shape

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
def make_model(n_in):
    inp = L.Input(name="inputs", shape=(n_in,))
    nh = 50
    x = L.Dense(nh, activation="relu", name="d1")(inp)
    x = L.Dense(nh, activation="relu", name="d2")(x)
    #x = L.Dense(nh, activation="relu", name="d3")(x)
    preds = L.Dense(4, activation="linear", name="preds")(x)
    
    model = M.Model(inp, preds, name="ANN")
    model.compile(loss="mean_absolute_error", optimizer="adam")
    return model
net = make_model(X.shape[1])
print(net.summary())

In [ ]:
oof = np.zeros(y.shape)
nets = []
EPOCHS  = 10
for idx in range(NFOLDS):
    print("FOLD:", idx)
    tr_idx, val_idx = folds[idx]
    ckpt = ModelCheckpoint(f"w{idx}.h5", monitor='val_loss', verbose=1, save_best_only=True,mode='min')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=3, min_lr=0.0005)
    es = EarlyStopping(monitor='val_loss', patience=6)
    reg = make_model(X.shape[1])
    reg.fit(X[tr_idx], y[tr_idx], epochs=EPOCHS, batch_size=30_000, 
            validation_data=(X[val_idx], y[val_idx]),
            verbose=1, callbacks=[ckpt, reduce_lr, es])
    reg.load_weights(f"w{idx}.h5")
    oof[val_idx] = reg.predict(X[val_idx], batch_size=50_000, verbose=1)
    nets.append(reg)
    gc.collect()
    #
#

In [ ]:
mae = mean_absolute_error(y, oof)
mse = mean_squared_error(y, oof, squared=False)
print("mae:", mae)
print("mse:", mse)

# Historical information to use in prediction time
bound_dt = pd.to_datetime("2021-01-01")
LAST = tr.loc[tr.EvalDate>bound_dt].copy()
LAST_MED_DF = MED_DF.loc[MED_DF.EvalYear==2021].copy()
LAST_MED_DF.drop("EvalYear", axis=1, inplace=True)
del tr
LAST.shape, LAST_MED_DF.shape, MED_DF.shape

In [ ]:
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

In [ ]:
"""players_cols = ['playerId', 'primaryPositionName']
rosters_cols = ['playerId', 'teamId', 'status']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances']

null = np.nan
true = True
false = False"""

In [ ]:
players_cols = ['playerId', 'primaryPositionName']
rosters_cols = ['playerId', 'teamId', 'status']
scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
       'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
       'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
       'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
       'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
       'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
       'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
       'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances',"gamePk"]
playertwitter_cols = ["playerId","numberOfFollowers","year_months"]
null = np.nan
true = True
false = False



env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

for (test_df, sample_prediction_df) in iter_test: # make predictions here
    
    sub = copy.deepcopy(sample_prediction_df.reset_index())
    sample_prediction_df = copy.deepcopy(sample_prediction_df.reset_index(drop=True))
    
    
    # creat dataset
    sample_prediction_df['playerId'] = sample_prediction_df['date_playerId']\
                                        .map(lambda x: int(x.split('_')[1]))
    # Dealing with missing values
    if test_df['rosters'].iloc[0] == test_df['rosters'].iloc[0]:
        test_rosters = pd.DataFrame(eval(test_df['rosters'].iloc[0]))
    else:
        test_rosters = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in rosters.columns:
            if col == 'playerId': continue
            test_rosters[col] = np.nan
            
    if test_df['playerBoxScores'].iloc[0] == test_df['playerBoxScores'].iloc[0]:
        test_scores = pd.DataFrame(eval(test_df['playerBoxScores'].iloc[0]))
    else:
        test_scores = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in scores.columns:
            if col == 'playerId': continue
            test_scores[col] = np.nan
    
    test_games = unpack_json(test_df["games"].iloc[0])
    test_standings = unpack_json(test_df["standings"].iloc[0])
    
            
    test_scores = test_scores.groupby('playerId').sum().reset_index()
    test = sample_prediction_df[['playerId']].copy()
    test = test.merge(players[players_cols], on='playerId', how='left')
    test = test.merge(test_rosters[rosters_cols], on='playerId', how='left')
    test = test.merge(test_scores[scores_cols], on='playerId', how='left')
    test = test.merge(test_games[games_cols], on="gamePk", how="left")
    test = test.merge(awards2[awards_cols], on=['playerId'], how='left')
    test = test.merge(player_target_stats, how='inner', left_on=["playerId"],right_on=["playerId"])
    test = test.merge(test_standings[standings_cols], on=["teamId"], how='left')
    test = test.merge(teamtwi[teamtwitter_cols], on=['teamId'], how='left')
    test = test.merge(hits_2020[hitscol], on=['playerId'], how='left')
    test = test.merge(game2020[gamescol], on=['playerId'], how='left')
    test["strdate"] = sample_prediction_df["date_playerId"].astype(str)
    test["date1"] = test["strdate"].str[0:8]
    twitter["strdate"] = twitter["date"].astype(str)
    twitter["year_months"] = twitter["strdate"].str[0:6].astype(int)
    test["year_months"] = test["strdate"].str[0:6].astype(int)
    test = test.merge(twitter[playertwitter_cols], on=['playerId'], how='left')
    #test["hitpergame"] = test["preseasonhits"] / test["preseasonpt"]

    
    #add feature
    test["strmonth"] = sample_prediction_df["date_playerId"].astype(str)
    test["hasTwitterAccount"] = test.playerId.isin(twitter.playerId)
    
    test["date2"] = pd.to_datetime(test["date1"].astype(str))
    test["month"] = test["strmonth"].str[5].median()
    test["youbi"] = test["date2"].dt.dayofweek
    test["allstar"] = 0
    test["regularseason"] = 1


    
    #label encoding
    test['label_playerId'] = test['playerId'].map(player2num)
    test['label_primaryPositionName'] = test['primaryPositionName'].map(position2num)
    test['label_teamId'] = test['teamId'].map(teamid2num)
    test['label_status'] = test['status'].map(status2num)
    test["label_daynight"] = test['dayNight'].map(daynight2num)
    test["label_seriesDescription"] = test["seriesDescription"].map(seriesDescription2num)
    test["gameType"] = test["gameType"].map(gameType2num)
    
    display(test)
    
    
    
    #test_X2 = test[feature_cols]
    #test_X3 = test[feature_cols]
    #test_X4 = test[feature_cols]
    
    # predict
    test_X = test[feature_cols]
    pred1 = model1.predict(test_X)
    #pred5 = model5.predict(test_X)
    #pred9 = model9.predict(test_X)
    test['target1'] = np.clip(pred1,0,100)
    test_X = test[feature_cols2]
    pred2 = model2.predict(test_X)
    #pred6 = model6.predict(test_X)
    #pred10 = model10.predict(test_X)
    test['target2'] = np.clip(pred2,0,100)
    test_X = test[feature_cols3]
    pred3 = model3.predict(test_X)
    #pred7 = model7.predict(test_X)
    #pred11 = model11.predict(test_X)
    test['target3'] = np.clip(pred3,0,100)
    
    test_X = test[feature_cols4]
    pred4 = model4.predict(test_X)
    #pred8 = model8.predict(test_X)
    #pred12 = model12.predict(test_X)
 
    
    # merge submission
    sample_prediction_df['target1'] = np.clip(pred1,0,100) #+ 0.2 * np.clip(pred5,0,100) + 0.2 * np.clip(pred9,0,100) 
    sample_prediction_df['target2'] = np.clip(pred2,0,100) #+ 0.2 * np.clip(pred6,0,100) + 0.2 * np.clip(pred10,0,100) 
    sample_prediction_df['target3'] = np.clip(pred3,0,100) #+ 0.2 * np.clip(pred7,0,100) + 0.2 * np.clip(pred11,0,100) 
    sample_prediction_df['target4'] = np.clip(pred4,0,100) #+ 0.2 * np.clip(pred8,0,100) + 0.2 * np.clip(pred12,0,100) 
    sample_prediction_df = sample_prediction_df.fillna(0.)
    yesterday_targets_test = sample_prediction_df
    yesterday_targets_test = yesterday_targets_test.rename(columns={'target1': 'yest_target1','target2': 'yest_target2','target3': 'yest_target3','target4': 'yest_target4'})
    del sample_prediction_df['playerId']
    #env.predict(sample_prediction_df)
    
    # TF summit
    # Features computation at Evaluation Date
    sub_fe, eval_dt = test_lag(sub)
    sub_fe = sub_fe.merge(LAST_MED_DF, on="playerId", how="left")
    sub_fe = sub_fe.fillna(0.)

    _preds = 0.
    for reg in nets:
        _preds += reg.predict(sub_fe[FECOLS + MEDCOLS]) / NFOLDS
    sub_fe[TGTCOLS] = np.clip(_preds, 0, 100)
    sub.drop(["date"]+TGTCOLS, axis=1, inplace=True)
    sub = sub.merge(sub_fe[["playerId"]+TGTCOLS], on="playerId", how="left")
    sub.drop("playerId", axis=1, inplace=True)
    sub = sub.fillna(0.)
    # Blending
    blend = pd.concat(
        [sub[['date_playerId']],
        (0.35*sub.drop('date_playerId', axis=1) + 0.65*sample_prediction_df.drop('date_playerId', axis=1))],
        axis=1
    )
    env.predict(blend)
    # Update Available information
    sub_fe["EvalDate"] = eval_dt
    #sub_fe.drop(MEDCOLS, axis=1, inplace=True)
    LAST = LAST.append(sub_fe)
    LAST = LAST.drop_duplicates(subset=["EvalDate","playerId"], keep="last")

In [ ]:
test["strmonth"].astype(str)
test["month"] = test["strmonth"].str[5].median()
test["month"]


In [ ]:
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

In [ ]:
unpack_json(test_df["games"].iloc[0])

In [ ]:
pd.concat(
    [sub[['date_playerId']],
    (sub.drop('date_playerId', axis=1) + sample_prediction_df.drop('date_playerId', axis=1)) / 2],
    axis=1
)

In [ ]:
test.dtypes

In [ ]:
sample_prediction_df.dtypes